In [250]:
# import pandas as pd
#  import matplotlib.pyplot as plt
from activation_functions import softmax
# from tensorflow.keras import activations
tf_sigmoid = activations.sigmoid
def sigmoid(X):
    return tf_sigmoid(X).numpy()

import numpy as np
import pandas as pd
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import re
from sklearn.metrics import mean_squared_error
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

# from tensorflow.keras.losses import CategoricalCrossentropy




In [307]:
class MY_LSTM:
    
    @staticmethod
    def my_build_model(hidden_size, input_size):
        lstm = MY_LSTM(hidden_size = hidden_size, input_size = input_size, optimizer=0)
        return lstm

    
    def __init__(self, hidden_size, input_size, optimizer):
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = 2
        self._optimizer = self.Adam
        self._loss_func = mean_squared_error
        self.USE_OPTIMIZER = True


        # input_size = 2
        # hidden_size = 1
        # z_size = 24
        z_size = self.hidden_size + self.input_size

        self.parameters = {
            "weights": {
                "Forget": self.__init_orthogonal(np.random.randn(z_size, self.hidden_size)),
                "Input": self.__init_orthogonal(np.random.randn(z_size, self.hidden_size)),
                "Candidate": self.__init_orthogonal(np.random.randn(z_size, self.hidden_size)),
                "Output": self.__init_orthogonal(np.random.randn(z_size, self.hidden_size)),
                "OutputSoftmax": self.__init_orthogonal(np.random.randn(self.hidden_size, self.output_size)),
            },
            "recurrent": {
                "Forget": self.__init_orthogonal(np.random.randn(self.hidden_size, self.hidden_size)),
                "Input": self.__init_orthogonal(np.random.randn(self.hidden_size, self.hidden_size)),
                "Candidate": self.__init_orthogonal(np.random.randn(self.hidden_size, self.hidden_size)),
                "Output": self.__init_orthogonal(np.random.randn(self.hidden_size, self.hidden_size))
            },
            "bias": {
                "Forget": np.ones((1, self.hidden_size)),
                "Input": np.ones((1, self.hidden_size)),
                "Candidate": np.ones((1, self.hidden_size)),
                "Output": np.ones((1, self.hidden_size)),
                "OutputSoftmax": np.ones((1, self.output_size)),

            }
        }

    @property
    def optimizer(self):
        return self._optimizer

    @optimizer.setter
    def optimizer(self, optimizer):
        self._optimizer = optimizer


    @property
    def loss_func(self):
        return self._loss_func

    @loss_func.setter
    def loss_func(self, loss_func):
        self.loss_func = loss_func


    def __init_orthogonal(self, param):
        """
        Initializes weight parameters orthogonally.
        This is a common initiailization for recurrent neural networks.

        Refer to this paper for an explanation of this initialization:
            https://arxiv.org/abs/1312.6120
        """
        if param.ndim < 2:
            raise ValueError("Only parameters with 2 or more dimensions are supported.")

        rows, cols = param.shape

        new_param = np.random.randn(rows, cols)

        if rows < cols:
            new_param = new_param.T

        # Compute QR factorization
        q, r = np.linalg.qr(new_param)

        d = np.diag(r, 0)
        ph = np.sign(d)
        q *= ph

        if rows < cols:
            q = q.T

        new_param = q

        return new_param

    @property
    def get_parameters(self):
        """
        Returns weights and biases as 2d array

        """
        return self.parameters

    def forward(self, input_data, prev_stm, prev_ltm):
        """
        Arguments:
            input_data -- your input data at timestep "t", numpy array of shape (n_x, m).
            prev_stm -- h at timestep "t-1", numpy array of shape (n_a, m)
            prev_ltm -- c at timestep "t-1", numpy array of shape (n_a, m)
        Returns:
            outputs -- prediction at timestep "t", numpy array of shape (n_v, m)
            
            Weight shape:
            All of the weights    shape (self.hidden_size, z_size)
            Except W_OutputSoftmax      (self.hidden_size, self.output_size)
            
            Data shapes:
            input_data            shape (1,1)
            prev_stm and prev_ltm shape (1,hidden_state)
            concat                shape is (1, z_size)
            concat.T              shape is (z_size, 1)
             - forget_gate        shape is (hidden_size, 1)
             - input_gate         shape is (hidden_size, 1)
             - candidate          shape is (hidden_size, 1)
             - next_ltm           shape is (1, hidden_size)
        """

        # Save a list of computations for each of the components in the LSTM
        
        concat = np.concatenate((prev_stm, input_data), axis=1)
#         print(f"Concat value is: \n", concat, "\nConcat.T value is: \n", concat.T)
        
#         Compute the forget gate
#         print("=======Computation of the forget gate:======")
#         print(f"W_forget:\n {self.parameters['weights']['Forget']}\nconcat:\n {concat.T}\nb_Forget: {self.parameters['bias']['Forget']}")
        z = np.dot(prev_stm, self.parameters['recurrent']['Forget'])
        y = np.dot(concat, self.parameters['weights']['Forget'])
#         print(f"np.dot(stm, forget): {z}")
#         print(f"np.dot(concat, forget): {y}")
#         print(f"bias: {self.parameters['bias']['Forget']}")
        
        forget_gate = sigmoid(np.dot(concat, self.parameters["weights"]["Forget"]) + np.dot(prev_stm, self.parameters["recurrent"]["Forget"]) + self.parameters["bias"]["Forget"])
#         print(f"Forget_gate value is: \n", forget_gate)
#         print("===========================================")


        # Compute the input gate
#         print(f"input_gate = concate.shape is {concat.shape} dot W_Input shape is {self.parameters['weights']['Input'].shape}")
        input_gate = sigmoid(np.dot(concat, self.parameters["weights"]["Input"]) + np.dot(prev_stm, self.parameters["recurrent"]["Input"]) + self.parameters["bias"]["Input"])
#         print(f"Input_gate value is: \n", input_gate)

        # Compute the candidate cell value
        candidate = np.tanh(np.dot(concat, self.parameters["weights"]["Candidate"]) + np.dot(prev_stm, self.parameters["recurrent"]["Candidate"]) + self.parameters["bias"]["Candidate"])
#         print(f"Candidate_gate value is: \n", candidate)

        # Compute the memory cell
#         print(f"==============next_ltm===========\n")
#         print(f"forget_gate * prev_ltm:\n {forget_gate * prev_ltm}")
#         print(f"input_gate * candidate: \n {input_gate * candidate}")
#         print(f"next_ltm:{forget_gate * prev_ltm + input_gate * candidate} ")
        next_ltm = forget_gate * prev_ltm + input_gate * candidate
#         print(f"next_ltm value is: \n", next_ltm)
#         print("==================================")


        # Compute the output gate
        output_gate = sigmoid(np.dot(concat, self.parameters["weights"]["Output"]) + np.dot(prev_stm, self.parameters["recurrent"]["Output"]) + self.parameters["bias"]["Output"])
#         print(f"Output_gate value is: \n", output_gate)

        # Compute the next hidden state
        next_stm = output_gate * np.tanh(next_ltm)
#         print(f"next_stm value is: \n", next_stm)

        forward_pass = {
            "Forget": forget_gate,
            "Input": input_gate,
            "Candidate": candidate,
            "Output": output_gate,
            "next_ltm": next_ltm,
            "next_stm": next_stm,
            "Concat_Input": concat
        }
        
        return forward_pass

    def __cross_entropy(self, predictions, targets, epsilon=1e-12):
        """
        Computes cross entropy between targets (encoded as one-hot vectors)
        and predictions.
        Input: predictions (N, k) ndarray
               targets (N, k) ndarray
        Returns: scalar
        """
        predictions = np.clip(predictions, epsilon, 1. - epsilon)
        N = predictions.shape[0]
        ce = -np.sum(targets * np.log(predictions + 1e-9)) / N
        return ce
    
    def calculate_loss(self, prediction, targets):
        print(f"============IN LOSS=============")
        print(f"prediction: {prediction}")
        print(f"targets: {targets}")
        x = self.loss_func(prediction, targets)
        print(x)
        return x

    def backward(self, forward_pass, prediction, targets):
        """
        Arguments:
        forward_pass -- dictionary:
                        "Forget_gate": forget_gate,
                        "Input_gate": input_gate,
                        "Candidate_gate": candidate,
                        "Output_gate": output_gate,
                        "next_ltm": next_ltm.T,
                        "next_stm": next_stm.T
                        
        targets -- your targets as a list of size m.
        Returns:
        loss -- crossentropy loss for all elements in output
        grads -- lists of gradients of every element in p
        
        Weight shape:
            All of the weights    shape (self.hidden_size, z_size)
            Except OutputSoftmax      (self.hidden_size, self.output_size)

        """

        gradients = {
            "weights": {
                "Forget": np.zeros_like(self.parameters["weights"]["Forget"]),
                "Input": np.zeros_like(self.parameters["weights"]["Input"]),
                "Candidate": np.zeros_like(self.parameters["weights"]["Candidate"]),
                "Output": np.zeros_like(self.parameters["weights"]["Output"]),
                "OutputSoftmax": np.zeros_like(self.parameters["weights"]["OutputSoftmax"]),
            },
            "recurrent": {
                "Forget": np.zeros_like(self.parameters["recurrent"]["Forget"]),
                "Input": np.zeros_like(self.parameters["recurrent"]["Input"]),
                "Candidate": np.zeros_like(self.parameters["recurrent"]["Candidate"]),
                "Output": np.zeros_like(self.parameters["recurrent"]["Output"]),
            }
#             "bias": {
#                 "b_Forget": np.zeros_like(self.parameters["bias"]["Forget"]),
#                 "b_Input": np.zeros_like(self.parameters["bias"]["Input"]),
#                 "b_Candidate": np.zeros_like(self.parameters["bias"]["Candidate"]),
#                 "b_Output": np.zeros_like(self.parameters["bias"]["Output"]),
#                 "b_OutputSoftmax": np.zeros_like(self.parameters["bias"]["OutputSoftmax"]),
#             }
        }

        # Set the next cell and hidden state equal to zero
#         print(f"==============PERFORMING BACKWARD===========\n")
#         print(f"INPUTS:\nforward_pass: \n{forward_pass}\ntargets: \n{targets}\nprediction: \n{prediction}")
#         print(f"\n==============END OF INPUTS=================\n")
        
#         print("\n===============PROCESSING------------------\n")
#         next_stm = np.zeros_like(forward_pass["next_stm"])  # h
#         next_ltm = np.zeros_like(forward_pass["next_ltm"])  # C
        next_ltm = forward_pass["next_ltm"]
        next_stm = forward_pass['next_stm']

        loss = 0
        # Compute the cross entropy
        t =1
#         for t in reversed(range(self.hidden_size)):
        loss += self.calculate_loss(prediction, targets)
#         print(f"[{t}] LOSS:  {loss}")

        
        # Compute the derivative of the relation of the hidden-state to the output gate
        dv = np.copy(prediction)
        dv[np.argmax(targets)] -= 1
#         print(f"dv (OUTPUT_GATE):  \n{dv}\n")
#         print(f"gradients[OutputSoftmax]:  \n{gradients['weights']['OutputSoftmax']}\n")
#         print(f"next_stm.T: \n{next_stm}")
#         print(f"np.dot(next_stm.T, dv): {np.dot(dv, next_stm)}")
        gradients["weights"]["OutputSoftmax"] += np.dot(dv, next_stm).T
        #gradients["bias"]["b_OutputSoftmax"] += dv.T 


        # Compute the derivative of the hidden state and output gate
        dh = np.dot(self.parameters["weights"]["OutputSoftmax"], dv).T
#         print("next_stm: ",next_stm)
#         print(f"dh:  \n{dh.T}\n")
        dh += next_stm
        
        
#         print(f"dh: {dh}")
#         print(f"next_ltm: {next_ltm}")
        do = dh * np.tanh(next_ltm)
#         print(f"do: {do}")
#         print(f"w_output: {forward_pass['Output']}")
        do = sigmoid(forward_pass["Output"]) * do
#         print(f"[{t}] do: \n{do}\n")
        # Update the gradients with respect to the output gate
        # =========================== ?????????DO WE NEED Concat_Input?????????????====================
#         print(f"do: {do.T.shape}")
#         print(f"Concat_input: {forward_pass['Concat_Input'].shape}")
        gradients["weights"]["Output"] += np.dot(do.T, forward_pass["Concat_Input"]).T
        gradients["recurrent"]["Output"] += np.dot(do.T, next_stm)

        #gradients["bias"]["b_Output"] += do
#         print(f"[{t}] gradients[Output] AFTER UPDATE:  \n{gradients['weights']['Output']}\n")


        # Compute the derivative of the cell state and candidate g
        dC = np.copy(next_ltm)
        dC += dh * forward_pass["Output"] * np.tanh(next_ltm)
#         print(f"[{t}] dC:  \n{dC}\n")

        dg = dC * forward_pass["Input"]
        dg = np.tanh(forward_pass["Candidate"]) * dg
#         print(f"[{t}] dg:  \n{dg}\n")


        # Update the gradients with respect to the candidate
#         print(f"[{t}] gradients[Candidate] BEFORE UPDATE:  \n{gradients['weights']['Candidate']}\n")
        # =========================== ?????????DO WE NEED Concat_Input?????????????====================
        gradients["weights"]["Candidate"] += np.dot(forward_pass["Concat_Input"].T, dg)
        gradients["recurrent"]["Candidate"] += np.dot(dg.T, next_stm)

        #gradients["bias"]["b_Output"] += dg
#         print(f"[{t}] gradients[Candidate] AFTER UPDATE:  \n{gradients['weights']['Candidate']}\n")


        # Compute the derivative of the input gate and update its gradients
        di = dC * forward_pass["Candidate"]
        di = sigmoid(forward_pass["Input"]) * di
#         print(f"[{t}] di:  \n{di}\n")

#         print(f"[{t}] gradients[Input] BEFORE UPDATE:  \n{gradients['weights']['Input']}\n")
        # =========================== ?????????DO WE NEED Concat_Input?????????????====================
        gradients["weights"]["Input"] += np.dot(forward_pass["Concat_Input"].T, di)
        gradients["recurrent"]["Input"] += np.dot(di.T, next_stm)

        #gradients["bias"]["b_Input"] += di
#         print(f"[{t}] gradients[Input] AFTER UPDATE:  \n{gradients['weights']['Input']}\n")


        # Compute the derivative of the forget gate and update its gradients
        df = dC * next_ltm
        df = sigmoid(forward_pass["Forget"]) * df
#         print(f"[{t}] df:  \n{df}\n")


#         print(f"[{t}] gradients[Forget] BEFORE UPDATE:  \n{gradients['weights']['Forget']}\n")
        # =========================== ?????????DO WE NEED Concat_Input?????????????====================
        gradients["weights"]["Forget"] += np.dot(forward_pass["Concat_Input"].T, df)
        gradients["recurrent"]["Forget"] += np.dot(df.T, next_stm)

        #gradients["bias"]["b_Forget"] += df
#         print(f"[{t}] gradients[Forget] AFTER UPDATE:  \n{gradients['weights']['Forget']}\n")


        # Compute the derivative of the input and update the gradients of the previous hidden and cell state
        dz = (np.dot(df, self.parameters["weights"]["Forget"].T) + np.dot(di, self.parameters["weights"]["Input"].T) + np.dot(
            dg, self.parameters["weights"]["Candidate"].T) + np.dot(do, self.parameters["weights"]["Output"].T))
#         print(f"[{t}] dz:  \n{dz}\n")

        dh_prev = dz[:self.hidden_size, :]
#         print(f"[{t}] dh_prev:  \n{dh_prev}\n")

        dC_prev = forward_pass["Forget"] * dC
#         print(f"[{t}] dC_prev:  \n{dC_prev}\n")


        # Clip gradients
#         print(f"=========\nGRADS BEFORE CLIP:  \n{grads}\n")

        grads = self.__clip_gradient_norm(gradients)

#         print(f"=========\nGRADS AFTER CLIP:  \n{grads}\n")


        return loss, grads

    def __clip_gradient_norm(self, grads, max_norm=0.25):
        """
        Clips gradients to have a maximum norm of `max_norm`.
        This is to prevent the exploding gradients problem.
        """
        # Set the maximum of the norm to be of type float
        max_norm = float(max_norm)
        total_norm = 0
        # Calculate the L2 norm squared for each gradient and add them to the total norm
        for gate, grad in grads["weights"].items():
            grad_norm = np.sum(np.power(grad, 2))
            total_norm += grad_norm
        total_norm = np.sqrt(total_norm)
        # Calculate clipping coeficient
        clip_coef = max_norm / (total_norm + 1e-6)
        # If the total norm is larger than the maximum allowable norm, then clip the gradient
        if clip_coef < 1:
            for gate, grad in grads["weights"].items():
                grad *= clip_coef
        return grads
    
    def Adam(self, parameters, gradients, learning_rate, t, beta1=0.9, beta2=0.999, epsilon=1e-8):
        """
        Adam optimization algorithm implementation.

        Arguments:
        parameters -- dictionary containing model parameters
        gradients -- dictionary containing gradients of model parameters
        learning_rate -- learning rate for the algorithm
        beta1 -- exponential decay rate for the first moment estimates (default: 0.9)
        beta2 -- exponential decay rate for the second moment estimates (default: 0.999)
        epsilon -- small constant to prevent division by zero (default: 1e-8)

        Returns:
        parameters -- updated model parameters
        """
#         print("===========INSIDE OPTIMIZER INPUTS=========\n")
#         print("parameters: \n", parameters)
#         print("gradients: \n", gradients)

        # Initialize the first and second moment estimates to zero
        first_moment = {}
        second_moment = {}

        # Initialize the parameters with zeros
        for param_name, param in parameters.items():
            first_moment[param_name] = np.zeros_like(param)
            second_moment[param_name] = np.zeros_like(param)

        # Perform Adam update for each parameter
        for param_name, param in parameters.items():
            # Update first moment estimate
            first_moment[param_name] = beta1 * first_moment[param_name] + (1 - beta1) * gradients[param_name]

            # Update second moment estimate
            second_moment[param_name] = beta2 * second_moment[param_name] + (1 - beta2) * np.square(gradients[param_name])

            # Bias correction
            first_moment_corrected = first_moment[param_name] / (1 - np.power(beta1, t))
            second_moment_corrected = second_moment[param_name] / (1 - np.power(beta2, t))

            # Update parameters
            parameters[param_name] -= learning_rate * first_moment_corrected / (np.sqrt(second_moment_corrected) + epsilon)

        return parameters

    
    def update_parameters(self, grads, t=1, lr=0.01):
        # Take a step
        parameters = self.get_parameters
#         print("===========IN UPDATE PARAMETERS========\n")
#         print("grads: ", grads)
#         print("parameters: ", parameters)
        if (self.USE_OPTIMIZER):
            updated_parameters = self.optimizer(parameters=parameters["weights"], gradients=grads["weights"], learning_rate=lr, t=t)
            updated_parameters = self.optimizer(parameters=parameters["recurrent"], gradients=grads["recurrent"], learning_rate=lr, t=t)

        else:
            for (_, parameter), (_, grad) in zip(parameters.items(), grads.items()):
                parameter -= lr * grad

In [187]:
# """
# TESTING OF FORWARD PASS WITH DUMMY VALUES
# """

# np.random.seed(1337)
# # Example data dimensions
# input_size = 1  # Number of features in the input
# hidden_state = 6  # Number of units in the hidden state/memory cell
# model = my_build_model(hidden_size=hidden_state, input_size = input_size)

# # Generate example input data, previous hidden state, and previous memory cell
# x = np.array([[9]])
# z = np.array([[11]])
# prev_stm = np.zeros((batch_size, hidden_state))
# prev_ltm = np.zeros((batch_size, hidden_state))
# print("\n=========Printing for MY_LSTM===============\n")
# print("INPUTS\n")
# print("(x) Input:\n", x)
# print("(prev_stm) {h} Previous hidden state:\n", prev_stm)
# print("(prev_ltm) {c} Previous memory cell:\n", prev_ltm)
# print("\n=========PERFORM FORWARD PASS==============\n")

# # Perform forward pass
# print("\n=========APPLY SOFTMAX (LAST LAYER OF LSTM)\n")
# forward_pass = model.forward(input_data=x, prev_stm=prev_stm, prev_ltm=prev_ltm)
# forward_pass = model.forward(input_data=z, prev_stm=forward_pass["next_stm"], prev_ltm=forward_pass["next_ltm"])
# reshaped_output = np.array(forward_pass['Output']).reshape(hidden_state)

# # Print the output
# print("===========OUTPUT============================")
# print("(next_ltm) {h} Next hidden state:\n", forward_pass["next_stm"])
# print("(next_stm) {c} Next memory cell:\n", forward_pass["next_ltm"])
# print("(Output_gate) RESHAPED:\n", reshaped_output)
# output_softmax = np.dot(forward_pass["next_stm"], model.parameters["weights"]["OutputSoftmax"]) + model.parameters["bias"]["OutputSoftmax"]
# output_softmax = softmax(output_softmax.reshape(2,1))
# print(f"softmax(Output)\n {output_softmax}")
# print("\n========END OF PRINTING FOR MY_LSTM========\n")

# print("\n========START OF TENSORFLOW LSTM===========\n")
# model_tf = build_model(x)
# print("Prediction is: ", model_tf.predict(x))
# print("\n========END OF TENSORFLOW LSTM=============\n")


# print("\n\n======START OF BACKWARDPROPAGATION=======\n")
# print(f"softmax(Output)\n {output_softmax}")
# model.backward(forward_pass=forward_pass, prediction=output_softmax, targets=[[1.0],[0.0]])
# print("\n\n======FINISH OF BACKWARDPROPAGATION======\n")

In [6]:
def build_model(X):
    model = Sequential()
    print(X.shape[1])
    model.add(Embedding(num_words, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out))
    model.add(Dropout(0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

    return model


In [315]:
def train_split(X, Y, test_size=0.8):
    length = int(len(X) * test_size)
    X_train = X[1:length]
    Y_train = Y[1:length]

    X_valid = X[length:]
    Y_valid = Y[length:]
    return X_train, X_valid, Y_train, Y_valid


def my_train(model, X, Y, hidden_size, input_size, num_epochs=2):
    X_train, X_valid, Y_train, Y_valid = train_split(X = X, Y = Y)
    training_loss, validation_loss = [], []

    for i in range(num_epochs):

        epoch_training_loss = 0
        epoch_validation_loss = 0
        
        sentence_size = len(X_valid) 
        it = 0
        for sentences, targets in zip(X_valid, Y_valid):            
            prev_stm = np.zeros((1, hidden_size))
            prev_ltm = np.zeros((1, hidden_size))
    
            for word in sentences:
                word = np.array([[word]])
                forward_pass = model.forward(word, prev_stm, prev_ltm)
                prev_stm = forward_pass["next_stm"]
                prev_ltm = forward_pass["next_ltm"]
            
            print("forward_pass['next_stm']", forward_pass["next_stm"])
            output_softmax = np.dot(forward_pass["next_stm"], model.parameters["weights"]["OutputSoftmax"]) + model.parameters["bias"]["OutputSoftmax"]
            output_softmax = output_softmax.reshape(2,1)
            loss = model.calculate_loss(output_softmax, targets)

            # Update loss
            epoch_validation_loss += loss
            it += 1
            print(f"Epoch {i}, {it} out of {sentence_size} loss: ", loss)
            break

        sentence_size = len(X_train)
        it = 0
        for inputs, targets in zip(X_train, Y_train):

            prev_stm = np.zeros((1, hidden_size))
            prev_ltm = np.zeros((1, hidden_size))

       
            for word in inputs:
                word = np.array([[word]])
                forward_pass = model.forward(word, prev_stm, prev_ltm)
                prev_stm = forward_pass["next_stm"]
                prev_ltm = forward_pass["next_ltm"]

            # Backward pass
            print("forward_pass['next_stm']", forward_pass["next_stm"])
            print("weights: \n", model.get_parameters)

            output_softmax = np.dot(forward_pass["next_stm"], model.parameters["weights"]["OutputSoftmax"]) + model.parameters["bias"]["OutputSoftmax"]
#             print("!##@!##!@#!@#!@#DEBUGGING!@#!@#!@#!@#!@#")
#             print("output_softmax: ", output_softmax)
#             print("type(output_softmax)", type(output_softmax))
            output_softmax = output_softmax.reshape(2,1)
#             print("RESHAPED output_softmax: ", output_softmax)

            loss, grads = model.backward(forward_pass=forward_pass, prediction=output_softmax, targets=targets)

            # Update parameters

            params = model.update_parameters(grads=grads)
            # Update loss
            #output_sentence = [dataset.idx_to_word[np.argmax(output)] for output in forward_pass["output_s"]]

            epoch_training_loss += loss
            it += 1
            print(f"Epoch {i+1}, {it} out of {sentence_size} loss: ", loss)

        # Save loss for plot
        training_loss.append(epoch_training_loss / len(X_train))
        validation_loss.append(epoch_validation_loss / len(X_valid))

        # Print loss every 2 epochs
        print(f'Epoch {i+1}, training loss: {training_loss[-1]}, validation loss: {validation_loss[-1]}')
        print(f'Input sentence {i}:')
        print(inputs)

        print(f'\nTarget sequence {i}:')
        print(targets)

        print('\nPredicted sequence:')
        output_softmax = np.dot(forward_pass["next_stm"], model.parameters["weights"]["OutputSoftmax"]) + model.parameters["bias"]["OutputSoftmax"]
        output_softmax = softmax(output_softmax.reshape(2,1))

        print(output_softmax)
    return training_loss, validation_loss

In [206]:
"""
====================FINAL=====================
Preparing the dataset
"""

def convert(x):
    """
    Coverting JSON to pandas dataframe

    """    
    ob = json.loads(x)
    for k, v in ob.items():
        if isinstance(v, list):
            ob[k] = ','.join(v)
        elif isinstance(v, dict):
            for kk, vv in v.items():
                ob['%s_%s' % (k, kk)] = vv
            del ob[k]
    return ob



def filter_data(data):
    """
    Converting into pandas dataframe and filtering only text and ratings given by the users
    """

    df = pd.DataFrame([convert(line) for line in data])
    df.drop(columns=df.columns.difference(['text','stars']),inplace=True)
    df.loc[:, ("sentiment")] = 0
    

#I have considered a rating above 3 as positive and less than or equal to 3 as negative.
    df.loc[:,'sentiment']=['pos' if (x>3) else 'neg' for x in df.loc[:, 'stars']]
    df.loc[:,'text'] = df.loc[:,'text'].apply(lambda x: x.lower())
    df.loc[:,'text'] = df.loc[:,'text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
    for idx,row in df.iterrows():
        df.loc[:,'text']= [x for x in df.loc[:,'text']]
    return df

def read_data():
    json_filename = 'review_mockup_500.json'
    with open(json_filename,'rb') as f:
        data = f.readlines()
    data = filter_data(data)
    tokenizer = Tokenizer(num_words = num_words, split=' ')
    tokenizer.fit_on_texts(data.loc[:,'text'].values)
    X = tokenizer.texts_to_sequences(data.loc[:,'text'].values)
    Y = pd.get_dummies(data['sentiment'], dtype=int).values   
    return X, Y

X, Y = read_data()

In [316]:
np.random.seed(1337)
batch_size = 1  # Number of training examples
input_size = 1  # Number of features in the input
hidden_size = 6  # Number of units in the hidden state/memory cell
model = MY_LSTM.my_build_model(hidden_size=hidden_size, input_size=input_size)
prev_stm = np.zeros((batch_size, hidden_size))
prev_ltm = np.zeros((batch_size, hidden_size))

my_train(model=model, X=X, Y=Y, hidden_size=hidden_size, input_size=input_size, num_epochs=5)

forward_pass['next_stm'] [[ 1.00000000e+000 -2.30998809e-166  7.15920300e-106  4.02756827e-046
  -1.36966942e-156 -1.00000000e+000]]
============IN LOSS=============
prediction: [[1.43898218]
 [1.37084445]]
targets: [0 1]
1.1040976529647388
Epoch 0, 1 out of 100 loss:  1.1040976529647388
forward_pass['next_stm'] [[ 1.00000000e+000 -0.00000000e+000  3.48128697e-250  1.12377236e-108
  -0.00000000e+000 -1.00000000e+000]]
weights: 
 {'weights': {'Forget': array([[-0.36976358,  0.28926954,  0.23064959, -0.33567422,  0.14610953,
        -0.31718936],
       [ 0.05027574, -0.64933983,  0.41365502, -0.43888968, -0.35605444,
        -0.27390194],
       [-0.01191829,  0.54362941,  0.1086134 , -0.04789943, -0.18503947,
        -0.61390835],
       [ 0.2041332 ,  0.06866587, -0.69129733, -0.68708789, -0.05913992,
         0.01042114],
       [-0.56908978,  0.19476206,  0.0067837 , -0.09364377, -0.64106089,
         0.45936122],
       [-0.70106888, -0.32756515, -0.26081895, -0.01685812,  0.442537

forward_pass['next_stm'] [[ 0.80123198 -0.1264261   0.32491283  0.44219147 -0.05049392 -0.72307106]]
weights: 
 {'weights': {'Forget': array([[-0.40975456,  0.24927128,  0.19077733, -0.37267927,  0.10611119,
        -0.35718437],
       [ 0.073462  , -0.62325915,  0.43044466, -0.42886455, -0.32993161,
        -0.24899049],
       [-0.04172053,  0.51369681,  0.08202466, -0.06193469, -0.21497637,
        -0.64379586],
       [ 0.17414706,  0.03866988, -0.72112358, -0.71265699, -0.08913655,
        -0.01957078],
       [-0.54764056,  0.21951098,  0.02161743, -0.08380363, -0.61622171,
         0.48269056],
       [-0.66107804, -0.28756692, -0.22094877,  0.02010417,  0.48253624,
        -0.1330222 ],
       [ 0.01978365,  0.18181478,  0.42682877, -0.49962765,  0.41534737,
         0.4143445 ]]), 'Input': array([[-0.0841371 ,  0.55808385, -0.19455486, -0.35092321,  0.11076936,
        -0.66855442],
       [-0.15424645, -0.50047036,  0.11505791, -0.52894174,  0.24122829,
         0.12616708],

forward_pass['next_stm'] [[ 5.61934471e-001 -1.07154106e-267  3.02733777e-171  1.91941091e-091
  -1.35697217e-253 -1.00000000e+000]]
weights: 
 {'weights': {'Forget': array([[-4.29748964e-01,  2.29271566e-01,  1.70786601e-01,
        -3.92406443e-01,  8.61114426e-02, -3.77183814e-01],
       [ 8.04839977e-02, -6.13469484e-01,  4.36345803e-01,
        -4.28412230e-01, -3.20120712e-01, -2.39393865e-01],
       [-5.14911886e-02,  5.03679057e-01,  7.23991729e-02,
        -6.65127643e-02, -2.24997721e-01, -6.53785499e-01],
       [ 1.54724605e-01,  1.87305775e-02, -7.38732180e-01,
        -7.22672264e-01, -1.09080329e-01, -3.94346513e-02],
       [-5.40999356e-01,  2.29261155e-01,  2.70869184e-02,
        -8.34215489e-02, -6.06446394e-01,  4.92213233e-01],
       [-6.41083591e-01, -2.67567206e-01, -2.00957977e-01,
         3.98331336e-02,  5.02535986e-01, -1.13022751e-01],
       [-2.16332198e-04,  1.61814777e-01,  4.06828826e-01,
        -5.19626363e-01,  3.95347368e-01,  3.94344505e-01]])

forward_pass['next_stm'] [[-9.98727581e-02 -2.92594270e-16  2.68112886e-10  1.51804156e-06
  -2.18660908e-15 -9.78032159e-01]]
weights: 
 {'weights': {'Forget': array([[-0.44552609,  0.20927283,  0.15099509, -0.41142571,  0.06611304,
        -0.39718251],
       [ 0.080484  , -0.61346948,  0.4363458 , -0.42841223, -0.32012071,
        -0.23939386],
       [-0.05149119,  0.50367906,  0.07239917, -0.06651276, -0.22499772,
        -0.6537855 ],
       [ 0.1547246 ,  0.01873058, -0.73873218, -0.72267226, -0.10908033,
        -0.03943465],
       [-0.54099936,  0.22926115,  0.02708692, -0.08342155, -0.60644639,
         0.49221323],
       [-0.62218054, -0.24756791, -0.18109195,  0.05939354,  0.52253527,
        -0.09302342],
       [-0.02020398,  0.14181478,  0.38682899, -0.53962504,  0.37534737,
         0.37434451]]), 'Input': array([[-0.12031376,  0.5381178 , -0.23403171, -0.39005261,  0.07080651,
        -0.70851561],
       [-0.15111231, -0.49404751,  0.11683411, -0.52829394,  0.24777

Epoch 1, 10 out of 399 loss:  0.4444711097328803
forward_pass['next_stm'] [[-1.54123159e-02 -1.08312134e-04  4.74684302e-03  2.28836138e-02
  -1.18067324e-04 -6.37622814e-01]]
weights: 
 {'weights': {'Forget': array([[-0.44445713,  0.22514735,  0.16144422, -0.40401279,  0.07951322,
        -0.38179195],
       [ 0.080484  , -0.61346948,  0.4363458 , -0.42841223, -0.32012071,
        -0.23939386],
       [-0.05149119,  0.50367136,  0.07239904, -0.06651277, -0.22500135,
        -0.65379199],
       [ 0.15472448,  0.01220791, -0.73903827, -0.72267515, -0.11376978,
        -0.04556158],
       [-0.54099936,  0.22926115,  0.02708692, -0.08342155, -0.60644639,
         0.49221323],
       [-0.61328091, -0.22756801, -0.16109556,  0.07912566,  0.54253504,
        -0.07302353],
       [-0.03981258,  0.12181478,  0.36682904, -0.55962081,  0.35534737,
         0.35434451]]), 'Input': array([[-0.1184675 ,  0.54868488, -0.22412622, -0.38197136,  0.08114261,
        -0.69801446],
       [-0.15111231

...

forward_pass['next_stm'] [[-6.77053763e-63 -0.00000000e+00  0.00000000e+00 -1.01134247e-48
  -0.00000000e+00 -1.48655289e-14]]
weights: 
 {'weights': {'Forget': array([[-0.42181869,  0.26424324,  0.20019515, -0.36798549,  0.1203439 ,
        -0.34073105],
       [ 0.08052526, -0.56366685,  0.47938077, -0.38914526, -0.26483489,
        -0.18056237],
       [-0.05346558,  0.44716634,  0.01943344, -0.11270875, -0.28707438,
        -0.7167767 ],
       [ 0.15647014,  0.05081556, -0.70364264, -0.68943157, -0.07405643,
        -0.00571644],
       [-0.54094249,  0.27956173,  0.07110504, -0.04401823, -0.54938066,
         0.55154023],
       [-0.56614304, -0.07859589, -0.01986523,  0.21774689,  0.69202133,
         0.07651566],
       [-0.09984627, -0.05818479,  0.18683032, -0.73961348,  0.1753474 ,
         0.17434452]]), 'Input': array([[-0.08488925,  0.58766077, -0.1861632 , -0.34482784,  0.1207082 ,
        -0.65814311],
       [-0.15066968, -0.44803004,  0.15760683, -0.48847962,  0.29435

forward_pass['next_stm'] [[-3.72070158e-06 -3.54146698e-41  2.36125256e-26 -4.75784638e-02
  -1.77376774e-35 -2.33838343e-01]]
weights: 
 {'weights': {'Forget': array([[-0.41778049,  0.2829411 ,  0.21952462, -0.34955213,  0.14016298,
        -0.32073537],
       [ 0.08057209, -0.5533793 ,  0.48961732, -0.37906039, -0.25284321,
        -0.16284455],
       [-0.05357889,  0.42905369,  0.0016884 , -0.12870557, -0.30678739,
        -0.73675508],
       [ 0.16374416,  0.07078623, -0.68361535, -0.66948267, -0.05368078,
         0.02424005],
       [-0.54089322,  0.29061758,  0.08196976, -0.03363961, -0.53452235,
         0.5708177 ],
       [-0.55832199, -0.05472615,  0.00575832,  0.24157732,  0.72514623,
         0.11631794],
       [-0.11378128, -0.10818405,  0.13683073, -0.78961262,  0.12534748,
         0.12434452]]), 'Input': array([[-0.07528231,  0.60664847, -0.16735876, -0.32592738,  0.14007666,
        -0.63821432],
       [-0.14431757, -0.4377788 ,  0.16773456, -0.47832854,  0.30484

forward_pass['next_stm'] [[-5.93488532e-031 -1.96182825e-247  2.42717951e-160 -1.47529841e-005
  -4.45392841e-213 -3.19537176e-002]]
weights: 
 {'weights': {'Forget': array([[-0.41778049,  0.30050761,  0.23811192, -0.33803   ,  0.15519887,
        -0.30077735],
       [ 0.08057209, -0.54344   ,  0.49958615, -0.36994725, -0.24306019,
        -0.1528443 ],
       [-0.05357889,  0.41907712, -0.00830776, -0.13823757, -0.31674052,
        -0.74932863],
       [ 0.16374416,  0.09058073, -0.66372742, -0.65155549, -0.03423038,
         0.04423748],
       [-0.54089322,  0.30051507,  0.09191705, -0.02505675, -0.52488478,
         0.58083049],
       [-0.55832199, -0.03234204,  0.03043118,  0.26355247,  0.74993522,
         0.14613064],
       [-0.11378129, -0.13818378,  0.10683088, -0.81961192,  0.09534762,
         0.09434452]]), 'Input': array([[-0.07528065,  0.62487916, -0.14954974, -0.31104172,  0.15612897,
        -0.61851005],
       [-0.14431757, -0.42782039,  0.17768201, -0.4685407 ,  0

forward_pass['next_stm'] [[ 0.31756918 -0.05726928  0.35473823  0.17507464 -0.21512639 -0.71778186]]
weights: 
 {'weights': {'Forget': array([[-0.4177803 ,  0.31835337,  0.2559584 , -0.33320761,  0.17420503,
        -0.28079348],
       [ 0.08057209, -0.54344   ,  0.49958615, -0.36994725, -0.24306019,
        -0.1528443 ],
       [-0.05357889,  0.41907711, -0.00830777, -0.13823757, -0.31674055,
        -0.74933354],
       [ 0.16374618,  0.11100459, -0.64271834, -0.64238769, -0.01339491,
         0.0754571 ],
       [-0.54089322,  0.30051507,  0.09191705, -0.02505675, -0.52488478,
         0.58083051],
       [-0.55796832,  0.00748915,  0.07029555,  0.28340205,  0.78984575,
         0.18612982],
       [-0.12060927, -0.1781833 ,  0.06683187, -0.85025591,  0.05534782,
         0.05434452]]), 'Input': array([[-0.07494027,  0.64342834, -0.13148352, -0.30220557,  0.17450375,
        -0.59866361],
       [-0.14431757, -0.42782039,  0.17768201, -0.4685407 ,  0.31470776,
         0.20396704],

KeyboardInterrupt: 